Розв'язати СЛАР двома методами:
* Гауса з вибором головного по стовпцях (прямий)
* Якобі (ітераційний)

In [146]:
import numpy as np

# Задамо СЛАР
a = np.array([
    [10, 1, 3, 1, 4],
    [2, 9, 1, 0, 2],
    [0, 2, 9, 1, 6],
    [1, 3, 1, 10, 2]
])

# кубічна норма матриці
def norm(a):
    r = -1
    for i in range(len(a)):
        sum = 0
        for j in range(len(a[i])):
            sum += abs(a[i][j])
        r = max(r, sum)
    return r

**Метод Гауса з вибором головного по стовпцях**

In [147]:
def gauss(a):
    swaps = 0
    maxElements = []

    # приведемо матрицю до верхнього трикутного вигляду
    for l in range(len(a)):
        maxElement = a[l][l]
        maxIndex = l
        for i in range(l + 1, len(a)):
            if a[i][l] > maxElement:
                maxElement = a[i][l]
                maxIndex = i
        maxElements.append(maxElement)
        if maxIndex != l:
            swaps += 1
            a[[l, maxIndex]] = a[[maxIndex, l]]
        
        m = np.identity(len(a))
        m[l][l] = 1/ a[l][l]
        for i in range(l + 1, len(a)):
            m[i][l] = -a[i][l] / a[l][l]
        a = np.matmul(m, a)

    # зворотній хід, приводимо до діагонального вигляду
    for i in range(len(a) - 1, 0, -1):
        for j in range(i - 1, -1, -1):
            a[j] = np.add(a[j], a[i] * -a[j][i])

    # обрахуємо визначник
    determinant = (-1) ** swaps * np.prod(maxElements)

    # масив розв'язків
    roots = []
    for i in range(len(a)):
        roots.append(a[i][-1])

    return roots, determinant

r, d = gauss(a)
print(f"gauss: {r}")
print(f"determinant: {d}")

gauss: [0.19069707511918563, 0.10952196881845122, 0.6329081303955677, 0.0847828888029893]
determinant: 7761.0


**Метод Якобі**

In [151]:
# перевірка на достатню умову збіжності
def meetsSufficientCondition(a):
    a = np.delete(a, len(a[0, :]) - 1, 1)
    for i in range(len(a)):
        sum = 0
        for j in range(len(a[i, :])):
            if i == j:
                continue
            sum += abs(a[i][j])
        if a[i][i] < sum:
            return False
    return True

epsilon = 10e-8

def jacobi(a):
    x = np.zeros((len(a), 1))
    while True:
        # виражаємо x_n через x_(n-1)
        xPrev = x
        x = np.zeros((len(a), 1))
        for i in range(len(a)):
            for j in range(len(a[i, :]) - 1):
                if i == j:
                    continue
                x[i][0] -= xPrev[j][0] * a[i][j]
            x[i][0] += a[i][len(a[i]) - 1]
            x[i][0] /= a[i][i]
        # умова припинення
        if norm(x - xPrev) < epsilon:
            break
    return x

if meetsSufficientCondition(a):
    r = jacobi(a)
    print(f"jacobi: {r}")
else:
    print("jacobi: check failed")

KeyboardInterrupt: 

**Число обумовленості**

In [149]:
def conditionNumber(a):
    # беремо коефіцієнти рівнянь
    c = np.delete(a, len(a[0, :]) - 1, 1)
    return norm(c) * norm(np.linalg.inv(c))

cn = conditionNumber(a)
print(f"conditioning number: {cn}")

conditioning number: 2.2825666795516044
